# Building Predictive Models

In [1]:
import pandas as pd
import os
import numpy as np

## Import Data

In [2]:
processed_data_path = os.path.join(os.path.pardir, "data", "processed")
train_file_path = os.path.join(processed_data_path, "train.csv")
test_file_path = os.path.join(processed_data_path, "test.csv")

In [3]:
train_df = pd.read_csv(train_file_path, index_col="PassengerId")
test_df = pd.read_csv(test_file_path, index_col="PassengerId")

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Survived            891 non-null    int64  
 1   Age                 891 non-null    float64
 2   Fare                891 non-null    float64
 3   FamilySize          891 non-null    int64  
 4   IsMother            891 non-null    int64  
 5   IsMale              891 non-null    int64  
 6   Deck_A              891 non-null    int64  
 7   Deck_B              891 non-null    int64  
 8   Deck_C              891 non-null    int64  
 9   Deck_D              891 non-null    int64  
 10  Deck_E              891 non-null    int64  
 11  Deck_F              891 non-null    int64  
 12  Deck_G              891 non-null    int64  
 13  Deck_Z              891 non-null    int64  
 14  Pclass_1            891 non-null    int64  
 15  Pclass_2            891 non-null    int64  
 16  Pclass_3

In [5]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Age                 418 non-null    float64
 1   Fare                418 non-null    float64
 2   FamilySize          418 non-null    int64  
 3   IsMother            418 non-null    int64  
 4   IsMale              418 non-null    int64  
 5   Deck_A              418 non-null    int64  
 6   Deck_B              418 non-null    int64  
 7   Deck_C              418 non-null    int64  
 8   Deck_D              418 non-null    int64  
 9   Deck_E              418 non-null    int64  
 10  Deck_F              418 non-null    int64  
 11  Deck_G              418 non-null    int64  
 12  Deck_Z              418 non-null    int64  
 13  Pclass_1            418 non-null    int64  
 14  Pclass_2            418 non-null    int64  
 15  Pclass_3            418 non-null    int64  
 16  Title

## Data Preparation

In [6]:
X = train_df.loc[:, "Age":].to_numpy().astype("float") # extract all columns from age onwards (ommitting Survived)
y = train_df["Survived"].ravel() # creates a one-dimension array

In [7]:
print(X.shape, y.shape)

(891, 32) (891,)


In [8]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) #random_state = seed value.
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(712, 32) (712,)
(179, 32) (179,)


In [9]:
# average survival in train and test
print("mean survival in train : {0:.3f}".format(np.mean(y_train)))
print("mean survival in test : {0:.3f}".format(np.mean(y_test)))

mean survival in train : 0.383
mean survival in test : 0.385


### Check Scikit-Learn Version

In [10]:
import sklearn

In [11]:
sklearn.__version__

'0.23.1'

## Baseline Model

In [12]:
from sklearn.dummy import DummyClassifier

In [13]:
model_dummy = DummyClassifier(strategy="most_frequent", random_state=0)

In [14]:
model_dummy.fit(X_train, y_train) #input data = X, output data = y

DummyClassifier(random_state=0, strategy='most_frequent')

In [15]:
print("score for baseline model : {0:.2f}".format(model_dummy.score(X_test, y_test)))

score for baseline model : 0.61


In [16]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

In [17]:
print("accuracy for baseline model : {0:.2f}".format(accuracy_score(y_test, model_dummy.predict(X_test))))

accuracy for baseline model : 0.61


In [18]:
print("confusion matrix for baseline model : \n {0}".format(confusion_matrix(y_test, model_dummy.predict(X_test))))

confusion matrix for baseline model : 
 [[110   0]
 [ 69   0]]


In [19]:
print("precision for baseline model : {0:.2f}".format(precision_score(y_test, model_dummy.predict(X_test))))
print("recall for baseline model : {0:.2f}".format(recall_score(y_test, model_dummy.predict(X_test))))

precision for baseline model : 0.00
recall for baseline model : 0.00


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## First Kaggle Submission

In [20]:
# converting to the matrix
test_X = test_df.to_numpy().astype("float")

In [21]:
# get predictions
predictions = model_dummy.predict(test_X)

In [22]:
df_submission = pd.DataFrame({"PassengerId": test_df.index, "Survived": predictions})

In [23]:
df_submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [24]:
submission_data_path = os.path.join(os.path.pardir, "data", "external")
submission_file_path = os.path.join(submission_data_path, "01_dummy.csv")

In [25]:
df_submission.to_csv(submission_file_path, index=False)

In [26]:
def get_submission_file(model, filename):
    # converting to the matrix
    test_X = test_df.to_numpy().astype("float")
    # make predictions
    predictions = model.predict(test_X)
    # submission dataframe
    df_submission = pd.DataFrame({"PassengerId": test_df.index, "Survived": predictions})
    # submission file
    submission_data_path = os.path.join(os.path.pardir, "data", "external")
    submission_file_path = os.path.join(submission_data_path, filename)
    # write to the file
    df_submission.to_csv(submission_file_path, index=False)

In [27]:
# get submission file
get_submission_file(model_dummy, "01_dummy.csv")

## Logistic Regression Model

In [28]:
from sklearn.linear_model import LogisticRegression

In [29]:
# create model
model_lr_1 = LogisticRegression(random_state=0, max_iter=1000)

In [30]:
# train model
model_lr_1.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=0)

In [31]:
# evaluate
print('score for logistic regression - version 1 : {0:.2f}'.format(model_lr_1.score(X_test, y_test)))

score for logistic regression - version 1 : 0.83


In [32]:
print("accuracy for logistic regression model : {0:.2f}".format(accuracy_score(y_test, model_lr_1.predict(X_test))))
print("confusion matrix for logistic regression model : \n {0}".format(confusion_matrix(y_test, model_lr_1.predict(X_test))))
print("precision for logistic regression model : {0:.2f}".format(precision_score(y_test, model_lr_1.predict(X_test))))
print("recall for logistic regression model : {0:.2f}".format(recall_score(y_test, model_lr_1.predict(X_test))))

accuracy for logistic regression model : 0.83
confusion matrix for logistic regression model : 
 [[95 15]
 [15 54]]
precision for logistic regression model : 0.78
recall for logistic regression model : 0.78


In [33]:
# model coefficients
model_lr_1.coef_

array([[-0.03273382,  0.00419841, -0.52723914,  0.63456442, -1.10972807,
        -0.01303799, -0.3013818 , -0.52886147,  0.40078109,  0.97689889,
         0.28529305, -0.28239305, -0.51909624,  0.59409093,  0.13624495,
        -0.7121334 ,  0.22437838,  1.10329746,  0.161041  , -1.55502215,
         0.74251117, -0.15451398, -0.50348939, -0.11868864, -0.03663297,
         0.00765927,  0.16586482,  0.12558798,  0.12224129, -0.22962679,
        -0.17372088,  0.19192335]])

## Second Kaggle Submission

In [34]:
get_submission_file(model_lr_1, "02_lr.csv")

## Hyperparameter optimization

In [36]:
model_lr = LogisticRegression(random_state=0)

In [37]:
from sklearn.model_selection import GridSearchCV

In [44]:
parameters = { 'C':[1.0, 10.0, 50.0, 100.0, 1000.0], 'penalty' : ['l1', 'l2']}
clf = GridSearchCV(model_lr, param_grid=parameters, cv=3)

In [45]:
clf.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logi

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linea

GridSearchCV(cv=3, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [1.0, 10.0, 50.0, 100.0, 1000.0],
                         'penalty': ['l1', 'l2']})

In [46]:
clf.best_params_

{'C': 10.0, 'penalty': 'l2'}

In [47]:
print("best score : {0:.2f}".format(clf.best_score_))

best score : 0.83


In [48]:
print("score for logistic regression - version 2 : {0:.2f}".format(clf.score(X_test, y_test)))

score for logistic regression - version 2 : 0.84


## Making third kaggle submission

In [49]:
get_submission_file(clf, "03_lr.csv")

## Feature normalization and standarization

In [50]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

### Feature Normalization

In [51]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [52]:
X_train_scaled[:,0].min(),X_train_scaled[:,0].max()

(0.0, 1.0)

In [53]:
X_test_scaled = scaler.transform(X_test)

### Feature Standarization

In [54]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Create model after standarization

In [56]:
model_lr = LogisticRegression()
parameters = {'C':[1.0, 10.0, 50.0, 100.0, 1000.0], 'penalty':["l1", "l2"]}
clf = GridSearchCV(model_lr, param_grid=parameters, cv=3)
clf.fit(X_train_scaled, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_v

GridSearchCV(cv=3, estimator=LogisticRegression(),
             param_grid={'C': [1.0, 10.0, 50.0, 100.0, 1000.0],
                         'penalty': ['l1', 'l2']})

In [57]:
clf.best_score_

0.8089978607476747

In [58]:
print("score for logistic regression - version 2 : {0:.2f}".format(clf.score(X_test_scaled, y_test)))

score for logistic regression - version 2 : 0.84


## Model Persistence

In [59]:
import pickle

In [95]:
model_file_path = os.path.join(os.path.pardir, "models", "lr_model.pkl")
scaler_file_path = os.path.join(os.path.pardir, "models", "lr_scaler.pkl")

In [96]:
model_file_pickle = open(model_file_path, 'wb')
scaler_file_pickle = open(scaler_file_path, 'wb')

In [97]:
pickle.dump(clf, model_file_pickle)
pickle.dump(scaler, scaler_file_pickle)

In [98]:
model_file_pickle.close()
scaler_file_pickle.close()

### load the persisted file

In [99]:
# open files
model_file_pickle = open(model_file_path, 'rb')
scaler_file_pickle = open(scaler_file_path, 'rb')

In [100]:
# load files
clf_loaded = pickle.load(model_file_pickle)
scaler_loaded = pickle.load(scaler_file_pickle)

In [101]:
#close
model_file_pickle.close()
scaler_file_pickle.close()

In [102]:
clf_loaded

GridSearchCV(cv=3, estimator=LogisticRegression(),
             param_grid={'C': [1.0, 10.0, 50.0, 100.0, 1000.0],
                         'penalty': ['l1', 'l2']})

In [103]:
scaler_loaded

StandardScaler()

In [104]:
# transform the test data using loaded scaler object
X_test_scaled = scaler_loaded.transform(X_test)
print("score for logistic regression - version 2 : {0:.2f}".format(clf_loaded.score(X_test_scaled, y_test)))

score for logistic regression - version 2 : 0.84
